# <font color = red> Simulation Analyze
This notebook records the parameters for Wright-Fisher simulations used to generate our test data sets, as well as commands for running infernce algorithms on the test data and compiling the results. 

## Contents
- ### [Libraries and variables](#lib)
- ### Data analyze
    - [Generation of test data through Wright-Fisher simulations](#sim)
    - [Collect multiple simulations](#collect)
    - [Finite sample data](#nsdt)

### <a id='lib'></a> Libraries and variables

In [1]:
print('This notebook was prepared using:')

import os
import sys
print('python version %s' % sys.version)

import numpy as np
print('numpy version %s' % np.__version__)

import pandas as pd
print('pandas version %s' % pd.__version__)

import math
from math import isnan

import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
print('matplotlib version %s' % matplotlib.__version__)

import re
import sys
import argparse
import scipy as sp
import random

from scipy import integrate
import scipy.interpolate as sp_interpolate
import statistics

from dataclasses import dataclass
import time as time_module

import json
from importlib import reload

import simulation as sim
import importlib

# GitHub directories
HIV_DIR = 'data/HIV'
MPL_DIR = 'src/MPL'
SIM_DIR = 'data/simulation'
FIG_DIR = 'figures'

This notebook was prepared using:
python version 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:26:08) [Clang 14.0.6 ]
numpy version 1.24.2
pandas version 1.5.3
matplotlib version 3.7.1


<!-- <a id=''></a> -->
### Generation of test data through Wright-Fisher simulations
The fitness model work like this:
$f_a = 1 + \sum_i^L s_i g_i^a$

This simulation begins with 4 random initial type, which only has 2 alleles (wild type and mutant type).

Wright-Fisher simulations are performed using simulation.py. The output of these simulations is saved for processing. 

In this part, we use python code to run mpl.

Benefial [0,1], neutral [2,3], delerious[4,5], time varying site (sin(t))[6,7], time varying site (sin(t))[8,9]

In [28]:
### parameter
importlib.reload(sim)

generations = 1000
fi_1 = np.zeros(generations+1)
fi_2 = np.zeros(generations+1)

for t in range(len(fi_1)):
    fi_1[t] = np.sin(t*2*np.pi/generations)*0.04
    fi_2[t] = np.cos(t*2*np.pi/generations)*0.04
    
pdata = {  
    'NUC':           ['A', 'T'],      # all possible alleles
    'dir':           'simple_new',        # directory of this simulation
    'xfile':         '0',             # output file name
    'output':        '',
    'seq_length':    10,              # sequence length
    'pop_size':      1000,            # population size
    'generations':   generations,     # number of total generations
    'totalT':        generations,     # generations used to infer
    'mut_rate':      1e-3,            # mutation rate
    'rec_rate':      1e-3,            # recombination rate
    'inital_state':  4,               # number of initial sub-population
    'bene':          [0,1],           # constant beneficial mutations sites
    'dele':          [4,5],           # constant deleterious mutations sites
    'p_1':           [6,7],           # time-varying mutations sites (sin)
    'p_2':           [8,9],           # time-varying mutations sites (cos)
    's_ben':         0.02,            # selection coefficient of beneficial mutations
    's_del':         -0.02,           # selection coefficient of deleterious mutations
    'fi_1':          fi_1,            # time-varying selection coefficient for individual site (sin)
    'fi_2':          fi_2,            # time-varying selection coefficient for individual site (cos)
    'gamma_s':       1,               # regularization - selection coefficients - constant part
    'gamma_2c':      100000,          # regularization - the time derivative of the selection coefficients
    'gamma_2tv':     200,             # regularization - the time derivative of the selection coefficients
    'bc_n':        True,            # True, using raw data; false, using finite sample noise
    }

In [29]:
reload(sim)

n_sim   = 100

# simulation
for k in range(n_sim):
    pdata['xfile']        = str(k)
    sim.simulate_simple(**pdata)

print('we have done %d times simulations'%n_sim)

we have done 100 times simulations


In [38]:
reload(sim)

for n in range(n_sim):
    pdata['xfile'] = str(n)
    pdata['bc_n']  = True
    sim.infer_simple(**pdata)
#     pdata['bc_n']  = False
#     sim.infer_simple(**pdata)

print('we have done inference for %d simulations '%n_sim)

we have done inference for 100 simulations 


In [39]:
simple_dir = pdata['dir']
const_num = 6
output = '_ls_11'
# Neumann boundary condition
f = open('%s/%s/mpl_collected%s.csv'%(SIM_DIR,simple_dir,output),'w')
f.write('trajectory,ns,delta_t')
for i in range(const_num):
    f.write(',sc_%d'%i)
f.write('\n')

for k in range(n_sim):
    name = str(k)
    data_full   = np.load('%s/%s/output%s/c_%s.npz'%(SIM_DIR,simple_dir,output,name), allow_pickle="True")
    sc_full     = data_full['selection']
    TimeVaryingSC = [np.average(sc_full[i]) for i in range(const_num)]
    f.write('%d,1000,1'%k)
    for i in range(const_num):
        f.write(',%f'%TimeVaryingSC[i])
    f.write('\n')
f.close()

In [11]:
const_num = 6

# Neumann boundary condition
f = open('%s/simple/mpl_collected.csv'%(SIM_DIR),'w')
f.write('trajectory,ns,delta_t')
for i in range(const_num):
    f.write(',sc_%d'%i)
f.write('\n')

for k in range(n_sim):
    name = str(k)
    data_full   = np.load('%s/simple/output/c_%s.npz'%(SIM_DIR,name), allow_pickle="True")
    sc_full     = data_full['selection']
    TimeVaryingSC = [np.average(sc_full[i]) for i in range(const_num)]
    f.write('%d,1000,1'%k)
    for i in range(const_num):
        f.write(',%f'%TimeVaryingSC[i])
    f.write('\n')
f.close()

# # Dirichlet boundary condition
# f = open('%s/simple/mpl_collected_d.csv'%(SIM_DIR),'w')
# f.write('trajectory,ns,delta_t')
# for i in range(const_num):
#     f.write(',sc_%d'%i)
# f.write('\n')

# for k in range(n_sim):
#     name = str(k)
#     data_full   = np.load('%s/simple/output_d/c_%s.npz'%(SIM_DIR,name), allow_pickle="True")
#     sc_full     = data_full['selection']
#     TimeVaryingSC = [np.average(sc_full[i]) for i in range(const_num)]
#     f.write('%d,1000,1'%k)
#     for i in range(const_num):
#         f.write(',%f'%TimeVaryingSC[i])
#     f.write('\n')
# f.close()

# print('collect all coefficients for %d simulations'%(n_sim))

collect all coefficients for 100 simulations


<a id='sim'></a>
### Generation of test data through Wright-Fisher simulations
The fitness model work like this:
$f_a = 1 + \sum_i^L s_i g_i^a + \sum_n^{N_p} s_n g_n^a$

This simulation begins with 4 random initial type, which only has 2 alleles (wild type and mutant type).

Wright-Fisher simulations are performed using simulation.py. The output of these simulations is saved for processing. 

In this part, we use python code to run mpl.

Benefial [0,1,2,3], delerious[16,17,18,19]

In [45]:
### parameter
importlib.reload(sim)

generations = 1000
fn = np.zeros(generations+1)
fi = np.zeros(generations+1)

for t in range(len(fn)):
    fi[t] = 0.04 - 0.08/generations * t
    fn[t] = 0.1  - 0.05/generations * t

trait_dir = 'trait'
pdata = {  
    'NUC':           ['A', 'T'],      # all possible alleles
    'dir':           trait_dir,       # directory of this simulation
    'xfile':         '0',             # output file name
    'seq_length':    20,              # sequence length
    'pop_size':      1000,            # population size
    'generations':   generations,     # number of total generations
    'totalT':        generations,     # generations used to infer
    'mut_rate':      1e-3,            # mutation rate
    'rec_rate':      1e-3,            # recombination rate
    'inital_state':  4,               # number of initial sub-population
    'n_ben':         4,               # number of beneficial mutations
    'n_del':         4,               # number of deleterious mutations
    's_ben':         0.02,            # selection coefficient of beneficial mutations
    's_del':         -0.02,           # selection coefficient of deleterious mutations
    'fi':            fi,              # time-varying selection coefficient for individual site
    'fn':            fn,              # time-varying selection coefficient for binary trait
    'escape_group':  [[12,15,17]],    # escape sites
    'escape_TF':     [[0,0,0]],       # wild type sequences for escape sites
    'trait_dis':     [[3,2]],         # distance between trait sites
    'p_sites':       [13,18],         # special sites (not escape sites but still time-varying)
    'x_thresh':      0.005,           # threshold for single allele frequency
    'gamma_s':       1,               # regularization - selection coefficients - constant part
    'gamma_2c':      100000,          # regularization - the time derivative of the selection coefficients
    'gamma_2tv':     200,             # regularization - the time derivative of the selection coefficients
    'bc_n':          True,            # True: Neumann boundary condition; False: Dirichlet boundary condition
    }

Use 3 files to restore the information about trait groups. ("traitsites": trait sites , "traitseq": TF sequences for trait sites,"traitdis":distance between 2 neighboring trait sites)

In [46]:
'Create the necessary files'
reload(sim)

n_sim   = 100

# # get random escape groups for 100 simulations
# escape_groups  = []
# special_groups = []
# for n in range(n_sim):
#     random_numbers   = random.sample(range(20), 5)
    
#     escape_group_raw = [random_numbers[:3]]
#     escape_group     = [sorted(sublist) for sublist in escape_group_raw]
#     escape_groups.append(escape_group)
    
#     p_sites_raw      = [random_numbers[3:]]
#     special_group    = [sorted(sublist) for sublist in p_sites_raw]
#     special_groups.append(special_group[0])
    
#     # trait sites 
#     f = open('%s/%s/traitsite/traitsite-%s.dat'%(SIM_DIR,trait_dir,n), 'w')
#     for i in range(len(escape_group)):
#         f.write('%s\n'%'\t'.join([str(ii) for ii in escape_group[i]]))
#     f.close()
    
#     # distance between 2 trait sites
#     f = open('%s/%s/traitdis/traitdis-%s.dat'%(SIM_DIR,trait_dir,n), 'w')
#     for i in range(len(escape_group)):
#         i_dis = []
#         for j in range(len(escape_group[i])-1):
#             i_dis.append(int(escape_group[i][j+1]-escape_group[i][j]))
#         f.write('%s\n'%'\t'.join([str(ii) for ii in i_dis]))
#     f.close()
    
#     # special sites
#     f = open('%s/%s/psites/psites-%s.dat'%(SIM_DIR,trait_dir,n), 'w')
#     f.write('%s\n'%'\t'.join([str(ii) for ii in special_group]))
#     f.close()

# # trait sequence 
# f = open('%s/%s/traitseq.dat'%(SIM_DIR,trait_dir), 'w')
# for i in range(2):
#     f.write('%s\n'%'\t'.join([str(0) for j in range(3)]))
# f.close()
    
# # save and load escape group information
# with open("%s/%s/escape_groups.dat"%(SIM_DIR,trait_dir), 'w') as file:
#     json.dump(escape_groups, file)
    
# with open("%s/%s/special_groups.dat"%(SIM_DIR,trait_dir), 'w') as file:
#     json.dump(special_groups, file)

In [47]:
importlib.reload(sim)

with open("%s/%s/escape_groups.dat"%(SIM_DIR,trait_dir), 'r') as file:
    escape_groups = json.load(file)

with open("%s/%s/special_groups.dat"%(SIM_DIR,trait_dir), 'r') as file:
    special_groups = json.load(file)
    
# simulation
for k in range(n_sim):
    pdata['xfile']        = str(k)
    pdata['escape_group'] = escape_groups[k]
    pdata['p_sites']      = special_groups[k]
#     sim.simulate_trait(**pdata)

print('we have done %d times simulations'%n_sim)

we have done 100 times simulations


Infer the results

In [52]:
importlib.reload(sim)

for n in range(n_sim):
    pdata['xfile']        = str(n)
    pdata['p_sites']      = special_groups[n]
    sim.infer_trait(**pdata)

print('we have done inference for %d simulations '%n_sim)

we have done inference for 100 simulations 


#### <a id='collect'></a> Collect multiple simulations
Create a csv file to store the results of all simulations.

In [56]:
nB = pdata['n_ben']
nD = pdata['n_del']
ne = len(pdata['escape_group'])

seq_length = pdata['seq_length']

with open("%s/%s/special_groups.dat"%(SIM_DIR,trait_dir), 'r') as file:
    special_groups = json.load(file)

out_dir = '_ll_12'
# Neumann boundary condition
f = open('%s/%s/mpl_collecte%s.csv'%(SIM_DIR,trait_dir,out_dir),'w')
f.write('trajectory,ns,delta_t')
for i in range(seq_length):
    f.write(',sc_%d'%i)
f.write('\n')

for k in range(100):
    name = str(k)
    data_full   = np.load('%s/%s/output%s/c_%s.npz'%(SIM_DIR,trait_dir,out_dir,name), allow_pickle="True")
    sc_full     = data_full['selection']
    TimeVaryingSC = [np.average(sc_full[i]) for i in range(seq_length)]
    p_sites = special_groups[k]
    f.write('%d,1000,1'%k)
    for i in range(seq_length):
        if i not in p_sites:
            f.write(',%f'%TimeVaryingSC[i])
        else:
            f.write(',nan')
    f.write('\n')
f.close()

In [30]:
nB = pdata['n_ben']
nD = pdata['n_del']
ne = len(pdata['escape_group'])

seq_length = pdata['seq_length']

with open("%s/%s/special_groups.dat"%(SIM_DIR,trait_dir), 'r') as file:
    special_groups = json.load(file)

# Neumann boundary condition
f = open('%s/%s/mpl_collected.csv'%(SIM_DIR,trait_dir),'w')
f.write('trajectory,ns,delta_t')
for i in range(seq_length):
    f.write(',sc_%d'%i)
f.write('\n')

for k in range(100):
    name = str(k)
    data_full   = np.load('%s/%s/output/c_%s.npz'%(SIM_DIR,trait_dir,name), allow_pickle="True")
    sc_full     = data_full['selection']
    TimeVaryingSC = [np.average(sc_full[i]) for i in range(seq_length)]
    p_sites = special_groups[k]
    f.write('%d,1000,1'%k)
    for i in range(seq_length):
        if i not in p_sites:
            f.write(',%f'%TimeVaryingSC[i])
        else:
            f.write(',nan')
    f.write('\n')
f.close()

# Dirichlet boundary condition
f = open('%s/%s/mpl_collected_d.csv'%(SIM_DIR,trait_dir),'w')
f.write('trajectory,ns,delta_t')
for i in range(seq_length):
    f.write(',sc_%d'%i)
f.write('\n')

for k in range(100):
    name = str(k)
    data_full   = np.load('%s/%s/output_d/c_%s.npz'%(SIM_DIR,trait_dir,name), allow_pickle="True")
    sc_full     = data_full['selection']
    TimeVaryingSC = [np.average(sc_full[i]) for i in range(seq_length)]
    p_sites = special_groups[k]
    f.write('%d,1000,1'%k)
    for i in range(seq_length):
        if i not in p_sites:
            f.write(',%f'%TimeVaryingSC[i])
        else:
            f.write(',nan')
    f.write('\n')
f.close()

print('collect all coefficients for %d simulations'%(k+1))

collect all coefficients for 100 simulations


#### <a id='nsdt'></a> Finite sample data inference

For one simulation, use different n_s and Δt to get the result

In [34]:
from importlib import reload
reload(sim)

ns_vals = [1000, 200, 100, 50, 20, 10]
dt_vals = [   1,   5,  10, 20, 50]
pdata['ns_vals'] = ns_vals
pdata['dt_vals'] = dt_vals

for k in range(n_sim):
    pdata['xfile'] = str(k)
    sim.py2c(**pdata)

In [13]:
reload(sim)

pdata['IF_raw'] = True
pdata['xfile']        = str(0)
pdata['p_sites']      = special_groups[0]
sim.infer_binary(**pdata)

In [46]:
pdata['IF_raw'] = False
for n in range(n_sim):
    pdata['p_sites']      = special_groups[n]
    for ns in ns_vals:
        for dt in dt_vals:
            if ns*dt>50:
                pdata['xfile'] = str(n)+'_ns'+str(ns)+'_dt'+str(dt)
                sim.infer_binary(**pdata)
                sim.infer_multiple(**pdata)

print('we have done inference for %d simulations with finite sample'%n_sim)

we have done inference for 100 simulations with finite sample


collect coefficients for all simulations and write the result into mpl_collected.csv.

In [47]:
# binary python 
f = open('%s/mpl_collected_nsdt.csv'%(SIM_DIR),'w')
f.write('trajectory,ns,delta_t')
for i in range(seq_length):
    f.write(',sc_%d'%i)
f.write('\n')

for k in range(100):
    for ns in ns_vals:
        for dt in dt_vals:
            if ns*dt>50:
                name = str(k)+'_ns'+str(ns)+'_dt'+str(dt)
                data_full   = np.load('%s/output/nsdt/c_%s.npz'%(SIM_DIR,name), allow_pickle="True")
                sc_full     = data_full['selection']
                TimeVaryingSC = [np.average(sc_full[i]) for i in range(seq_length)]
                p_sites = special_groups[k]
                f.write('%d,%d,%d'%(k,ns,dt))
                for i in range(seq_length):
                    if i not in p_sites:
                        f.write(',%f'%TimeVaryingSC[i])
                    else:
                        f.write(',nan')
                f.write('\n')
f.close()

print('collect all coefficients for %d simulations'%(k+1))

collect all coefficients for 100 simulations


In [48]:
# multiple python 
f = open('%s/mpl_collected_nsdt_multiple.csv'%(SIM_DIR),'w')
f.write('trajectory,ns,delta_t')
for i in range(seq_length):
    f.write(',sc_%d'%i)
f.write('\n')

for k in range(100):
    for ns in ns_vals:
        for dt in dt_vals:
            if ns*dt>50:
                name = str(k)+'_ns'+str(ns)+'_dt'+str(dt)
                data_full   = np.load('%s/output_multiple/nsdt/c_%s.npz'%(SIM_DIR,name), allow_pickle="True")
                sc_full     = data_full['selection']
                TimeVaryingSC = [np.average((sc_full[2*i+1]-sc_full[2*i])) for i in range(seq_length)]
                p_sites = special_groups[k]
                f.write('%d,%d,%d'%(k,ns,dt))
                for i in range(seq_length):
                    if i not in p_sites:
                        f.write(',%f'%TimeVaryingSC[i])
                    else:
                        f.write(',nan')
                f.write('\n')
f.close()

print('collect all coefficients for %d simulations'%(k+1))

collect all coefficients for 100 simulations


In [53]:
import sklearn as sk
from sklearn.metrics import roc_auc_score

nB = pdata['n_ben']
nD = pdata['n_del']

fB = pdata['s_ben']
fD = pdata['s_del']


true_ben = [1 if i in                        range(nB) else 0 for i in range(seq_length)]
true_del = [1 if i in range(seq_length-nD, seq_length) else 0 for i in range(seq_length)]

coefs = ['sc_%d' % j for j in range(seq_length)]

df              = pd.read_csv('%s/mpl_collected_nsdt.csv'%SIM_DIR, memory_map=True)

# difference between inferred coefficients and true coefficients
for i in range(seq_length):
    if   true_ben[i]: df['d_sc%d' % i] = df['sc_%d' % i] - fB
    elif true_del[i]: df['d_sc%d' % i] = df['sc_%d' % i] - fD


# # AUROC for beneficial and deleterious mutation
# df['AUROC_ben'] = pd.Series(data=[roc_auc_score(true_ben, np.array(df.iloc[i][coefs])) for i in range(len(df))])
# df['AUROC_del'] = pd.Series(data=[roc_auc_score(true_del,-np.array(df.iloc[i][coefs])) for i in range(len(df))])


df.to_csv('%s/mpl_collected_extended.csv'%SIM_DIR)


In [53]:
def getMutantS(sVec,nVec):
    muVec    = -np.ones(seq_length)
    x_length = 0
    for i in range(seq_length):
        allele_count = np.zeros(len(sVec))
        allele_count = [np.sum([(sVec[t][k][i]==1)*nVec[t][k] for k in range(len(sVec[t]))]) for t in range(len(sVec))]
        if max(allele_count) / np.sum(nVec[0]) >= 0:
            muVec[i] = x_length
            x_length += 1
    return x_length,muVec

def get_allele_frequency(sVec,nVec,eVec,muVec):

    x  = np.zeros((len(nVec),x_length))           # single allele frequency
    xx = np.zeros((len(nVec),x_length,x_length))  # pair allele frequency
    for t in range(len(nVec)):
        pop_size_t = np.sum([nVec[t]])
        # individual locus part
        for i in range(seq_length):
            aa = int(muVec[i])
            if aa != -1:
                x[t,aa] = np.sum([sVec[t][k][i] * nVec[t][k] for k in range(len(sVec[t]))]) / pop_size_t
            for j in range(int(i+1), seq_length):
                bb = int(muVec[j])
                if bb != -1:
                    xx[t,aa,bb] = np.sum([sVec[t][k][i] * sVec[t][k][j] * nVec[t][k] for k in range(len(sVec[t]))]) / pop_size_t
                    xx[t,aa,bb] = np.sum([sVec[t][k][i] * sVec[t][k][j] * nVec[t][k] for k in range(len(sVec[t]))]) / pop_size_t
        # escape part
        for n in range(ne):
            aa      = x_length-ne+n
            x[t,aa] = np.sum([eVec[t][k][n] * nVec[t][k] for k in range(len(sVec[t]))]) / pop_size_t
            for m in range(int(n+1), ne):
                bb          = x_length-ne+m
                xx[t,aa,bb] = np.sum([eVec[t][k][n] * eVec[t][k][m] * nVec[t][k] for k in range(len(sVec[t]))]) / pop_size_t
                xx[t,bb,aa] = np.sum([eVec[t][k][n] * eVec[t][k][m] * nVec[t][k] for k in range(len(sVec[t]))]) / pop_size_t
            for j in range(seq_length):
                bb = int(muVec[j])
                if bb != -1:
                    xx[t,bb,aa] = np.sum([sVec[t][k][j] * eVec[t][k][n] * nVec[t][k] for k in range(len(sVec[t]))]) / pop_size_t
                    xx[t,aa,bb] = np.sum([sVec[t][k][j] * eVec[t][k][n] * nVec[t][k] for k in range(len(sVec[t]))]) / pop_size_t
    return x,xx


In [56]:
### Average Covariance
# load data
data         = np.loadtxt("%s/example/example-0.dat"%(SIM_DIR))
seq_length   = 20
ne           = 1

# Obtain sequence data and frequencies
sVec,nVec,eVec = sim.getSequence(data,escape_group)
x_length,muVec = getMutantS(sVec,nVec)
x_length      += ne

# Allele frequencies
x,xx         = get_allele_frequency(sVec,nVec,eVec,muVec) 

# Covariance matrix
covariance_n = sim.diffusion_matrix_at_t(x, xx,x_length)
covariance   = np.swapaxes(covariance_n, 0, 2)



In [60]:
covariance_a = np.zeros((len(covariance),len(covariance[0])))
for i in range(len(covariance)):
    for j in range(len(covariance[i])):
        covariance_a[i,j] = np.average(covariance[i,j,:])
    

In [61]:
print(covariance_a[0,0])
print(covariance[0,0,:])


0.14262595409181636
[0.       0.002991 0.003984 0.004975 0.004975 0.005964 0.005964 0.010879
 0.018639 0.021516 0.024375 0.0196   0.016711 0.0196   0.017676 0.0196
 0.020559 0.025324 0.0291   0.032844 0.044791 0.050191 0.0384   0.039319
 0.0475   0.043884 0.034704 0.035631 0.045696 0.039319 0.042064 0.042975
 0.046599 0.059031 0.057279 0.069375 0.075276 0.071071 0.070224 0.071071
 0.064239 0.0651   0.072759 0.0651   0.058156 0.054636 0.044791 0.059904
 0.052864 0.039319 0.050191 0.0475   0.055519 0.059904 0.064239 0.084351
 0.09     0.093184 0.084351 0.086784 0.088396 0.083536 0.085975 0.081079
 0.076944 0.088396 0.093184 0.090799 0.108624 0.1131   0.116044 0.113839
 0.113839 0.133036 0.144375 0.148876 0.1539   0.158796 0.16     0.154519
 0.149511 0.159399 0.156364 0.1716   0.1659   0.162384 0.1716   0.190464
 0.198924 0.201159 0.203775 0.207975 0.211975 0.214279 0.220071 0.223104
 0.2304   0.226591 0.227199 0.227799 0.230956 0.226591 0.235359 0.240784
 0.240975 0.245644 0.244524 0.242

In [ ]:
### orthogonal diagonalization

import scipy.linalg as la


A1 = np.array([[2, -1], [1, 3]])
b1 = np.array([1, 0])
A2 = np.array([[1, 2], [-1, 4]])
b2 = np.array([0, 1])


u_prime_0 = 1
u_prime_1 = 2


t_start = 0
t_end = 1
num_intervals = 100
dt = (t_end - t_start) / num_intervals
t_half = int(num_intervals / 2)


def orthogonal_diagonalization(A):
    eigvals, eigvecs = la.eig(A)
    Q = eigvecs
    D = np.diag(eigvals)
    return Q, D

Q1, D1 = orthogonal_diagonalization(A1)
Q2, D2 = orthogonal_diagonalization(A2)


b1_transformed = Q1.T @ b1
b2_transformed = Q2.T @ b2


def finite_difference_method(D, b_transformed, dt, num_intervals, initial_condition):
    x = np.zeros((D.shape[0], num_intervals + 1))
    x[:, 0] = initial_condition
    for i in range(num_intervals):
        x[:, i + 1] = x[:, i] + dt * (D @ x[:, i] + b_transformed)
    return x


x1 = finite_difference_method(D1, b1_transformed, dt, t_half, np.array([0, 0]))
x2_initial_condition = Q1 @ x1[:, t_half] - Q2 @ x1[:, t_half] + (u_prime_0 * dt) * np.array([1, 0])
x2 = finite_difference_method(D2, b2_transformed, dt, t_half, x2_initial_condition)


u1 = Q1 @ x1
u2 = Q2 @ x2


u = np.hstack((u1[:, :t_half], u2[:, 1:]))


In [47]:
importlib.reload(sim)

# inference
pdata['xpath']    = 'jobs'
for n in range(n_sim):
    pdata['xfile'] = str(n)
    sim.infer_binary(**pdata)

In [61]:
name = 'example'
seq_length = 20


data_full   = np.load('%s/example/c_%s_new.npz'%(SIM_DIR,name), allow_pickle="True")
sc_full     = data_full['selection']
TimeVaryingSC = [np.average(sc_full[i]) for i in range(seq_length)]
print(TimeVaryingSC)
print(sc_full[0][:10])

data_full   = np.load('%s/example/c_%s_multiple.npz'%(SIM_DIR,name), allow_pickle="True")
sc_full     = data_full['selection']
TimeVaryingSC = [np.average(sc_full[2*i+1]-sc_full[2*i]) for i in range(seq_length)]
# TimeVaryingSC = [np.average(TimeVaryingSC[i]) for i in range(seq_length)]
print(TimeVaryingSC)
print(sc_full[1][:10]-sc_full[0][:10])


[0.016443938397976306, 0.01839057910090518, 0.016954357072892424, 0.007952794805819752, -0.00010838909471058121, -0.0026846497716975664, -0.004497659231641359, -0.0017100268149185499, -0.0011023367612964941, 0.0030902011440607784, -0.004467527252911888, -0.0012500813284500273, 0.0026862670416833655, 0.0011594286845331102, -0.005469852620909327, -0.006463447223312858, -0.011758975207705382, -0.01230974345262893, -0.01610816459542871, -0.013210852761702311]
[0.01641512 0.0164152  0.01641528 0.01641535 0.01641541 0.01641546
 0.0164155  0.01641556 0.01641563 0.01641568]
[0.02032294086114561, 0.023286064722280557, 0.020527547093306203, 0.009924747185991682, -0.0019360454401982218, -0.0029482882722519344, -0.005155539471279753, -0.0025383234078301333, -0.0010977250380106993, 0.003161800079556894, -0.005034400960466398, -0.0020927823901811284, 0.0023538538704306056, 0.0011164760936466826, -0.005916639688167631, -0.008831599304957, -0.015112407528142737, -0.016692912960450574, -0.0209886300236

In [27]:
############################################################################
############################## Function ####################################
def getSequence(history,escape_group):
    sVec      = []
    nVec      = []
    eVec      = []

    temp_sVec   = []
    temp_nVec   = []
    temp_eVec   = []

    times       = []
    time        = 0
    times.append(time)

    ne          = len(escape_group)

    for t in range(len(history)):
        if history[t][0] != time:
            time = history[t][0]
            times.append(int(time))
            sVec.append(temp_sVec)
            nVec.append(temp_nVec)
            eVec.append(temp_eVec)
            temp_sVec   = []
            temp_nVec   = []
            temp_eVec   = []

        temp_nVec.append(history[t][1])
        temp_sVec.append(history[t][2:])

        if ne > 0: # the patient contains escape group
            temp_escape = np.zeros(ne, dtype=int)
            for n in range(ne):
                for nn in range(len(escape_group[n])):
                    index = escape_group[n][nn] + 2
                    if history[t][index] != 0:
                        temp_escape[n] = 1
                        break
            temp_eVec.append(temp_escape)

        if t == len(history)-1:
            sVec.append(temp_sVec)
            nVec.append(temp_nVec)
            eVec.append(temp_eVec)

    return sVec,nVec,eVec

def getMutantS(sVec,nVec):
    # use muVec matrix to record the index of time-varying sites(after throwing out weak linkage sites)
    muVec = -np.ones((seq_length, q)) # default value is -1, positive number means the index
    x_length  = 0
    for i in range(seq_length):
        allele_uniq = [int(j) for j in np.unique(sVec[:][:][i])] # all possible alleles in site i
        for allele in allele_uniq:
            # throw out the alleles with low frequency
            allele_count = np.zeros(len(sVec))
            allele_count = [np.sum([(sVec[t][k][i]==allele)*nVec[t][k] for k in range(len(sVec[t]))]) for t in range(len(sVec))]
            if max(allele_count) / np.sum(nVec[0]) >= x_thresh:
                muVec[i][int(allele)] = x_length
                x_length += 1
    return x_length,muVec

############################################################################
####################### Inference (binary case) ############################
# obtain raw data
data         = np.loadtxt("%s/example/example.dat"%(SIM_DIR))
escape_group = [[12,15,17]]
ne           = len(escape_group)

# obtain sequence data and frequencies
sVec,nVec,eVec = getSequence(data,escape_group)
x_length,muVec = getMutantS(sVec,nVec)
x_length += ne

In [36]:
import time as time_module
q = 2
x_thresh = 0.01
x = np.zeros((len(nVec),x_length))  # pair allele frequency
xx = np.zeros((len(nVec),x_length,x_length))  # pair allele frequency

start_time = time_module.time()
'''code 1'''

'''code 1'''
end_time = time_module.time()
print(f"Execution time: {end_time - start_time} seconds")


start_time = time_module.time()
'''code 2'''

'''code 2'''
end_time = time_module.time()
print(f"Execution time: {end_time - start_time} seconds")




Execution time: 0.5478200912475586 seconds
[ 750.  250. 1000.    0.  750.  250.  750.  250. 1000.    0.  750.  250.
  500.  500.  750.  250.  750.  250.  750.  250.  750.  250.  750.  250.
 1000.    0.  750.  250. 1000.    0.  750.  250.  500.  500. 1000.    0.
  500.  500. 1000.    0.    0.]
Execution time: 1.6201841831207275 seconds
[ 750.  250. 1000.    0.  750.  250.  750.  250. 1000.    0.  750.  250.
  500.  500.  750.  250.  750.  250.  750.  250.  750.  250.  750.  250.
 1000.    0.  750.  250. 1000.    0.  750.  250.  500.  500. 1000.    0.
  500.  500. 1000.    0.    0.]


### Dealing with the output
collect coefficients for all simulations and write the result into mpl_collected.csv.

In [14]:
nB = pdata['n_ben']
nD = pdata['n_del']
ne = len(pdata['escape_group'])
p_sites = pdata['p_sites']
seq_length = pdata['seq_length']
dirr = '2n1d-10-500'

f = open('%s/mpl_collected_%s.csv'%(SIM_DIR,dirr),'w')
f.write('trajectory')
for i in range(seq_length):
    if i not in p_sites:
        f.write(',sc_%d'%i)
f.write('\n')

for k in range(100):
    name = 'example-2n1d-'+str(k)
    data_full   = np.load('%s/%s/c_%s.npz'%(SIM_DIR,dirr,name), allow_pickle="True")
    sc_full     = data_full['selection']
    TimeVaryingSC = [np.average((sc_full[2*i+1]-sc_full[2*i])) for i in range(seq_length)]
    f.write('%d'%k)
    for i in range(seq_length):
        if i not in p_sites:
            f.write(',%f'%TimeVaryingSC[i])
    f.write('\n')
f.close()

print('collect all coefficients for %d simulations'%(k+1))

collect all coefficients for 100 simulations


In [101]:
for i in range(100):
    name = 'example-2n1d-'+str(i)
    sc = np.loadtxt('%s/2n1d/sc-%s-const.dat'%(SIM_DIR,name))
    if len(sc) != 21:
        print(i)

In [121]:
from imp import reload
reload(sim)

'''1N2D'''

generations = 500
fP = np.zeros(generations+1)
for t in range(len(fP)):
    fP[t] = 0.05 - 0.05/generations * t

pdata = {  
    'NUC':           ['A', 'C'],      # all possible alleles                              
    'name':          'example',       # output file name
    'seq_length':    20,              # sequence length
    'pop_size':      1000,            # population size
    'generations':   generations,     # number of total generations
    'totalT':        500,             # generations used to infer
    'mutation_rate': 1e-3,            # mutation rate
    'inital_state':  4,               # number of initial sub-population
    'n_ben':         4,               # number of beneficial mutations
    'n_del':         4,               # number of deleterious mutations
    's_ben':         0.02,            # selection coefficient of beneficial mutations
    's_del':         -0.02,           # selection coefficient of deleterious mutations
    'fP':            fP,              # time-varying selection coefficient
    'escape_group':  [[12,16,19]],      # escape sites
    'p_sites':       [13,18],         # special sites (not escape sites but still time-varying)
    'x_thresh':      0.001,           # threshold for single allele frequency
    'gamma_s':       10,              # regularization - selection coefficients - constant part
    'gamma_2c':      1000000,         # regularization - the time derivative of the selection coefficients
    'gamma_2tv':     200,             # regularization - the time derivative of the selection coefficients
    }

for i in range(100):
    pdata['name'] = 'example-1n2d-'+str(i)
    sim.simulate(**pdata)
    sim.infer_const(**pdata)
    sim.infer_binary(**pdata)
    sim.infer_multiple(**pdata)

In [122]:
nB = pdata['n_ben']
nD = pdata['n_del']
ne = len(pdata['escape_group'])
p_sites = pdata['p_sites']
seq_length = pdata['seq_length']

f = open('%s/mpl_collected_1n2d.csv'%SIM_DIR,'w')
f.write('trajectory')
for i in range(seq_length):
    if i not in p_sites:
        f.write(',sc_%d'%i)
f.write('\n')

for k in range(100):
    name = 'example-1n2d-'+str(k)
    data_full   = np.load('%s/1n2d/c_%s.npz'%(SIM_DIR,name), allow_pickle="True")
    sc_full     = data_full['selection']
    TimeVaryingSC = [np.average((sc_full[2*i+1]-sc_full[2*i])) for i in range(seq_length)]
    f.write('%d'%k)
    for i in range(seq_length):
        if i not in p_sites:
            f.write(',%f'%TimeVaryingSC[i])
    f.write('\n')
f.close()

print('collect all coefficients for %d simulations'%(k+1))

collect all coefficients for 100 simulations
